Importing necessary libraries

In [28]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from PIL import Image
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

Defining necessary parameters

In [29]:
DATA_DIR = '../../Data/'         # Path to your dataset folder
IMG_SIZE = (224, 224)                # VGG16 input size
BATCH_SIZE = 32
EPOCHS = 10

Set up ImageDataGenerators for training and validation
Includes augmentation for training set

In [30]:
train_gen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,             # 80/20 train/val split
    horizontal_flip=True,
    rotation_range=10,
    zoom_range=0.1
)

# Load training data
train_data = train_gen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

# Load validation data
val_data = train_gen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

Found 8012 images belonging to 2 classes.
Found 2003 images belonging to 2 classes.


Load the VGG16 base model (pretrained on ImageNet)

In [31]:
base_model = VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

# Freeze all convolutional layers so only our custom head is trained
for layer in base_model.layers:
    layer.trainable = False

Add custom classification head

In [32]:
x = base_model.output
x = GlobalAveragePooling2D()(x)        # Converts conv output to 1D
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)  # Binary classification

# Combine base + custom layers
model = Model(inputs=base_model.input, outputs=predictions)

Compile the model

In [33]:
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

Train the model

In [44]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS
)

ImportError: Could not import PIL.Image. The use of `load_img` requires PIL.

Evaluate the model

In [ ]:
val_data.reset()
y_true = val_data.classes                        # True labels
y_pred = model.predict(val_data, verbose=1)      # Probabilities
y_pred_binary = (y_pred > 0.5).astype(int).reshape(-1)  # Convert to 0/1

# Classification metrics
print("Classification Report:")
print(classification_report(y_true, y_pred_binary, target_names=['benign', 'malignant']))

print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred_binary))

Optional: Plot accuracy and loss curves

In [ ]:
plt.figure(figsize=(14, 5))

# Accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train acc')
plt.plot(history.history['val_accuracy'], label='Val acc')
plt.title("Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()

# Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train loss')
plt.plot(history.history['val_loss'], label='Val loss')
plt.title("Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()

plt.tight_layout()
plt.show()